In [1]:
#To extract page numbers(pb), div2 label, and catnum from Misc XML file and save as csv.

#<div3 rend="deleted"> to work out

#Extract div3 when there is no catnum within each div2 labels
#Excludes Indexes
#To ignore digits from <add rend="del"> and <add rend="pencil"> which occurs once with catnum 2082
#from <catnum>, remove: -; ____; ?; .; and a tick in front of a number.
#Excludes non-Sloane cat entries.

from lxml import etree
import re
import csv

In [2]:
path = '/Users/deborahleem/Documents/1_PhD/XMLSloane2021/output_person_place_misc_cleaned_up_25042021.xml'
doc = etree.parse(open(path))
root = doc.getroot()

In [3]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
        #print('removing', etree.tostring(e, pretty_print=True))
        if e.text == '-1861' or e.text == '2094.' or e.text == '-382.':
            continue
        remove_element(e)


/Users/deborahleem/scripts/venvs/deb_1.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  """


In [4]:
#Catnums that need to be excluded as they are not part of Sloane
#<pb xml:id="EPF108491027v"/>, Miscellanies, 2108, 2109, 2110, 2111
#<pb xml:id="EPF108491177v"/>, Antiquities, 1,2
#<pb xml:id="EPF108491178v"/>, Antiquities, 1,2,3,4
#<pb xml:id="EPF108491179v"/>, Antiquities, 4,5,6,7,8,1
#<pb xml:id="EPF108491180v"/>, Antiquities, 2,3
#<pb xml:id="EPF108491181v"/>, Antiquities, 4,5
#<pb xml:id="EPF108491182v"/>, Antiquities, 1,2,3,4,5
#<pb xml:id="EPF108491183v"/>, Antiquities, 6,7,8,9,10,11,12
#<pb xml:id="EPF108491184v"/>, Antiquities, 13,14,1,2,3,4,5,1
#<pb xml:id="EPF108491186v" type="Bronzes"/>, Antiquities, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
#<pb xml:id="EPF108491187v" type="Bronzes"/>, Antiquities, 20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
#<pb xml:id="EPF108491188v"/>, Antiquities, 40,41,42,43,44,45,46,47,48,1,2,3,4,5,6,7,8,9
#<pb xml:id="EPF108491189v"/>, Antiquities, 10,11,12,1,2,3,4,5,6
#<pb xml:id="EPF108491190v"/>, Antiquities, 7,8,9,10,11

In [5]:
excluded = {
  "EPF108491027v": ["2108","2109","2110","2111"], # "Miscellanies"
  "EPF108491177v": ["1","2"], # "Antiquities"
  "EPF108491178v": ["1","2","3","4"], # "Antiquities"
  "EPF108491179v": ["4","5","6","7","8","1"], # "Antiquities"
  "EPF108491180v": ["2","3"], # "Antiquities"
  "EPF108491181v": ["4","5"], # "Antiquities"
  "EPF108491182v": ["1","2","3","4","5"], # "Antiquities"
  "EPF108491183v": ["6","7","8","9","10","11","12"], # "Antiquities"
  "EPF108491184v": ["13","14","1","2","3","4","5","1"], # "Antiquities"
  "EPF108491186v": ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"], # "Antiquities"
  "EPF108491187v": ["20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39"], # "Antiquities"
  "EPF108491188v": ["40","41","42","43","44","45","46","47","48","1","2","3","4","5","6","7","8","9"], # "Antiquities"
  "EPF108491189v": ["10","11","12","1","2","3","4","5","6"], # "Antiquities"
  "EPF108491190v": ["7","8","9","10","11"], # "Antiquities"
}


In [6]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return ''.join(label.itertext()).strip()
            return
        element = element.getparent()

In [7]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
#         print(element.tag)
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()
                
# for catnum in root.iterfind('.//{*}catnum'):
#     print(find_pb_xml_id(catnum))


In [8]:
#metamark not consistenly used.

In [9]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

results = []
for catnum in root.iterfind('.//{*}catnum'):
    text = "".join(catnum.itertext())
    text = cleanup_catnum(text)
    label = find_div2_label(catnum)
    if label.startswith('Index'):
        continue
    pb_xml_id = find_pb_xml_id(catnum)
    if pb_xml_id in excluded:
        if text in excluded[pb_xml_id]:
#             print('exclude', pb_xml_id, text)
            continue
#         else:
#             print('do not exclude', pb_xml_id, text)
    
    results.append([text, label, pb_xml_id])
    #print(etree.tostring(catnum))
    #print([text, label])

In [10]:
# Find div3s without catnums inside, not in index
#There are many Div3 without catnum on recto so only retrieve ones on verso
for div3 in root.iterfind('.//{*}div3'):
    catnums_inside = list(div3.iterfind('.//{*}catnum'))
    if len(catnums_inside) > 0:
        continue
    label = find_div2_label(div3)
    if label.startswith('Index'):
        continue
    pb_xml_id = find_pb_xml_id(div3)
    if pb_xml_id in excluded:
         continue
    if pb_xml_id.endswith('v'):
        results.append(['no catnum', label, pb_xml_id])
#     print(div3)
#     print(etree.tostring(div3))
#     break


In [11]:
# results0 = results

In [12]:
#Showing all changes.
# [(results0[i], results[i]) for i in range(len(results)) if results0[i] != results[i]]

In [13]:
# All entries that are not just digits and start
[[x, results.index(x)] for x in results if not x[0].replace('*', '').isdigit()]

[[['No 1', 'Miscellanies.', 'EPF108491009v'], 238],
 [['1933 No 7', 'Miscellanies.', 'EPF108491010v'], 259],
 [['1933 No 8', 'Miscellanies.', 'EPF108491010v'], 260],
 [['1933 No 9', 'Miscellanies.', 'EPF108491010v'], 261],
 [['1933 No 10', 'Miscellanies.', 'EPF108491010v'], 262],
 [['1933 No 11', 'Miscellanies.', 'EPF108491010v'], 263],
 [['1933 No 12', 'Miscellanies.', 'EPF108491010v'], 264],
 [['1933 No 13', 'Miscellanies.', 'EPF108491010v'], 265],
 [['1933 No 14', 'Miscellanies.', 'EPF108491010v'], 266],
 [['1933 No 15', 'Miscellanies.', 'EPF108491010v'], 267],
 [['1933 No 16', 'Miscellanies.', 'EPF108491010v'], 268],
 [['1933 No 17', 'Miscellanies.', 'EPF108491010v'], 269],
 [['1933 No 18', 'Miscellanies.', 'EPF108491010v'], 270],
 [['1933 No 20', 'Miscellanies.', 'EPF108491010v'], 271],
 [['1933 No 21', 'Miscellanies.', 'EPF108491010v'], 272],
 [['1933 No 22', 'Miscellanies.', 'EPF108491010v'], 273],
 [['1933 No 25', 'Miscellanies.', 'EPF108491010v'], 274],
 [['1933 No 26', 'Misce

In [14]:
with open('all_catnums_and_their_div2_labels_July_2021.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['catnum', 'label','pageid'])
    for catnum, label, pageid in results:
        writer.writerow([catnum, label, pageid])

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('all_catnums_and_their_div2_labels_July_2021.csv', header=None, names=['catnum', 'div2_label', 'page_nu'])
# exclude no catnum values

# split into types - with catnums and without catnumbs across all labels
df_with_catnum = df[~(df.catnum == 'no catnum')]
df_no_catnum = df[(df.catnum == 'no catnum')]

In [17]:
len(df_with_catnum.catnum.unique())

2176

In [18]:
len(df_no_catnum)

21

In [19]:
# Find all rows which have catnums and are of type Misc

df_with_catnum_misc = df_with_catnum[(df_with_catnum.div2_label == 'Miscellanies.')]

In [20]:
# Find all rows which have catnums and are of type Misc and are unique

len(df_with_catnum_misc.catnum.unique())

2147

In [21]:
# Find all rows which have catnums and are of type Misc (total)
len(df_with_catnum_misc.catnum)

2165

In [22]:
# Find all rows which dont have catnums and are of type Misc
len(df_no_catnum[df_no_catnum.div2_label == 'Miscellanies.'])

16

In [23]:
# rows with catnums across label
df_with_catnum.groupby('div2_label').count()

,catnum,page_nu
div2_label,,
Agate cups botles spoons &c.,303,303
Agate handles &c.,241,241
Antiquities.,1170,1170
Impressions of Seals &c.,260,260
Mathematicall instruments &c,56,56
Miscellanies.,2165,2165
Pictures &c.,413,413
label,1,1


In [24]:
# rows without catnums across label
df_no_catnum.groupby('div2_label').count()

,catnum,page_nu
div2_label,,
Antiquities.,5,5
Miscellanies.,16,16


In [25]:
# all rows
df.groupby('div2_label').count()

,catnum,page_nu
div2_label,,
Agate cups botles spoons &c.,303,303
Agate handles &c.,241,241
Antiquities.,1175,1175
Impressions of Seals &c.,260,260
Mathematicall instruments &c,56,56
Miscellanies.,2181,2181
Pictures &c.,413,413
label,1,1


In [26]:
# rows with catnums across label that are duplicates. Note doesnt contain the first non duplicate row of the group
df_with_catnum['x'] = df_with_catnum.duplicated(keep='first', subset=['catnum', 'div2_label'])
# df_no_dupes = df[df.x == True]
# df.catnum.duplicated(keep='first')
# df.head()
# df[df.x == 'True']
# df['x'] == 'True'
dupes = df_with_catnum[df_with_catnum.x == True] 
dupes.groupby('div2_label').count()

/Users/deborahleem/scripts/venvs/deb_1.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,catnum,page_nu,x
div2_label,,,
Agate handles &c.,1,1,1
Antiquities.,18,18,18
Miscellanies.,18,18,18
Pictures &c.,2,2,2


In [27]:
dupes

,catnum,div2_label,page_nu,x
240,2,Miscellanies.,EPF108491009v,True
241,3,Miscellanies.,EPF108491009v,True
242,4,Miscellanies.,EPF108491009v,True
243,6,Miscellanies.,EPF108491009v,True
513,2046,Miscellanies.,EPF108491019v,True
556,2064,Miscellanies.,EPF108491021v,True
573,2068,Miscellanies.,EPF108491022v,True
888,542,Miscellanies.,EPF108491051,True
1096,396,Miscellanies.,EPF108491079,True
1268,909,Miscellanies.,EPF108491099,True


In [28]:
len(dupes)

39

In [29]:
# test to see if data is correct
dupes[dupes.div2_label == 'Pictures &c.']

,catnum,div2_label,page_nu,x
3964,368,Pictures &c.,EPF108491268,True
3988,382,Pictures &c.,EPF108491269,True


In [30]:
df[(df.div2_label == 'Agate cups botles spoons &c.') & (df.catnum == '1')]

,catnum,div2_label,page_nu
4306,1,Agate cups botles spoons &c.,EPF108491305


In [31]:
df.groupby('div2_label').count()

,catnum,page_nu
div2_label,,
Agate cups botles spoons &c.,303,303
Agate handles &c.,241,241
Antiquities.,1175,1175
Impressions of Seals &c.,260,260
Mathematicall instruments &c,56,56
Miscellanies.,2181,2181
Pictures &c.,413,413
label,1,1


In [ ]:


# is catnum empty?
df[df.catnum == None]
df[df.catnum.isnull()]
df[df.catnum == '']

In [ ]:
df[df.div2_label.isnull()]

In [ ]:
df[df.div2_label == '']

In [ ]:
df.sort_values('div2_label', ascending=False)

In [ ]:
#total number of catnum = 4608
len(df)

In [ ]:
len(df.div2_label.unique())

In [ ]:
df.div2_label.unique()

In [ ]:
#unique number of pages = 369 including Index. Without 382

len(df.page_nu.unique())

In [ ]:

misc = df[df.div2_label == 'Miscellanies.']

In [ ]:
len(misc.catnum)

In [ ]:
len(misc.catnum.unique())

In [ ]:
misc[misc.catnum.duplicated(keep=False)].sort_values('catnum')

In [ ]:
misc[misc.catnum.duplicated(keep=False)].sort_values('catnum').to_csv("dupicate_catnum_each_div2.csv")

In [ ]:
antq = df[df.div2_label == 'Antiquities.']

In [ ]:
len(antq.catnum)

In [ ]:
len(antq.catnum.unique())

In [ ]:
antq[antq.catnum.duplicated(keep=False)].sort_values('catnum')

In [ ]:
cups = df[df.div2_label == 'Agate cups botles spoons &c.']

In [ ]:
len(cups.catnum)

In [ ]:
len(cups.catnum.unique())

In [ ]:
seals = df[df.div2_label == 'Impressions of Seals &c.']

In [ ]:
len(seals.catnum)

In [ ]:
len(seals.catnum.unique())

In [ ]:
pics = df[df.div2_label == 'Pictures &c.']

In [ ]:
len(pics.catnum)

In [ ]:
len(pics.catnum.unique())

In [ ]:
pics[pics.catnum.duplicated(keep=False)].sort_values('catnum')

In [ ]:
#to get text from div3: .join(div3.itertext()).strip()

In [ ]:
#to get text from div3: .join(div3.itertext()).strip()